Data extraction and analysis from social media platform Youtube by generating Youtube API key

# Installing Required Clients, Libraries and Setting up API Key

**The Project entails utilizing the YouTube API through Python for data extraction related to the "avatar movie." It involves setting up the API key, extracting video details, and obtaining statistics for the top 50 videos in the US. Analysis includes sorting videos by top comments, retrieving comments, exporting data in JSON format, and calculating the likes vs views ratio for top videos with the most comments.**

**Install the Google API python client**

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

**Importing Necessary Libraries**

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from pprint import pprint
import warnings
import os
import pandas as pd
import json

**Defining variables for storing data and initializing caching.**

In [ ]:
Video_Ids = []
Video_Statistics = []
Video_Comments = []
DisabledComments_Videos = []

In [ ]:
#Set up the Query String in advance
query_string = 'avatar movie'

**Building the Youtube API Using the API_Key Created.**

In [ ]:
# Executed with my API KEY, Please Pass the appropriate API Key below ,When Running:
api_Key = ''
# Building the Youtube API
try:
    youtube_API = build('youtube', 'v3', developerKey=api_Key)

except Exception as e:
    print("An UnExpected Error had Occured While Building the Youtube API: ", e)

# Defining the User Defined Functions:

**getTopFiftyVideosDataOrderedByRelevance:**

**User Defined Function to Get the Top 50 Videos Related Information Ordered By Relevance in US Region**

In [ ]:
def getTopFiftyVideosDataOrderedByRelevance(query):
    try:
        # Getting Top 50 Videos Ordered By Relevance in US Region.
        search_response = youtube_API.search().list(
            q=query,
            type='video',
            part='id,snippet',
            regionCode='US',
            maxResults=50,
            order='relevance'
        ).execute()

        # Gathering the Necessary Information.
        for search_result in search_response.get('items', []):
            video_id = search_result['id']['videoId']

            # Appends the Retrieved VideoId's into Video_Ids Cache DataList.
            Video_Ids.append({'video_id': video_id})

        return Video_Ids

    except HttpError as e:
            if e.resp.status == 403 and 'quotaExceeded' in e.content.decode('UTF-8'):
                print("Quota exceeded. Please Retry With the Provided Alternative API KEY...")
            else:
                print(f'An Error has Occured While Retrieving Top 50 Videos Ordered By Relevance in US Region: {e}')

**getVideoSpecificStatistics :**


**Retrieves the statistics for each video, including the video title, total number of likes, comments, and views.**

In [ ]:
def getVideoSpecificStatistics(video_Id):
    try:
        # Getting Statistics for Each Video.
        search_response = youtube_API.videos().list(
                part='snippet,statistics',
                id = video_Id,
            ).execute()

        # Get the Required Information.
        video_title = search_response['items'][0]['snippet']['title']
        likes_count= int(search_response['items'][0]['statistics'].get('likeCount', 0))
        comment_count = int(search_response['items'][0]['statistics'].get('commentCount',0))
        view_count = int(search_response['items'][0]['statistics'].get('viewCount', 0))

        # Append the Retrieved Data into the Video_Statistics Cache DataList.
        Video_Statistics.append({'Video_Id':video_Id, 'Video_title':video_title,'Likes_count':likes_count,
                                 'Comments_count':comment_count, 'Views_Count':view_count})

        # Print the Required Output if necessary
        # print(f"Video Id :{video_Id}, Number of Likes :{likes_count},Number of Comments: {comment_count}, Number of Views: {view_count}")

        return Video_Statistics

    except HttpError as e:
        print(f'An Error has Occured While Retrieving Statistics : {e}')

 **getCommentsFromVideo :**


**This function retrieves comments for a given video ID using the YouTube API, structures the data into JSON format, and appends it to a cache variable named Video_Comments.**

In [ ]:
def getCommentsFromVideo(videoId):
    # JSON structure to store VideoId and Respective Comments.
    videoId_and_comments = {'Video_ID': videoId, 'Video_Comments': []}
    try:
        # Getting the Comments For Each VideoId.
        search_response = youtube_API.commentThreads().list(
                  part='snippet',
                  videoId = videoId,
                  textFormat='plainText',
                  maxResults=25
                  ).execute()

        # Getting the Items from the Response
        items = search_response.get('items', [])

        # Gathering Required Comment Info for all the Comments.
        for prop in items:
            # Append the Retrieved Data Into the JSON.
            videoId_and_comments['Video_Comments'].append({'CommentInfo':prop})

        # Append the JSON into the Video_Comments Cache DataList.
        Video_Comments.append(videoId_and_comments)
        return Video_Comments

    except HttpError as e:
         print(f'An Error has Occured While Retrieving the Comments for VideoID:{videoId}, With Exception :{e}')

**getVideoIdsWithDisabledComments :**


**Retrieves the Videos with Disabled Comments, So that we can     Filter Out those Video's to Mitigate 403 Errors .**

In [ ]:
def getVideoIdsWithDisabledComments(videoId):
    try:
        # Getting the VideodIs's With the Disabled Comments.
        search_response = youtube_API.commentThreads().list(
                    part='snippet',
                    videoId = videoId,
                    textFormat='plainText',
                    maxResults=10
                    ).execute()

    except HttpError as e:
            # Get the Error Reason.
            error_reason = e.error_details[0]['reason']
            if error_reason == 'commentsDisabled':
                # Add the VideoId with commentsDisabled Error to the DisabledComments_Videos List.
                DisabledComments_Videos.append({'Videoid':videoId})
                # To Mitigate the 403 Errors Stopping the Execution ,Ignore the Errors.
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore", category=UserWarning, message="Encountered 403 Forbidden with reason 'commentsDisabled'")

    return DisabledComments_Videos

**getDataFrame :**

**Creates a DataFrame for the Provided List of Data.**

In [ ]:
def getDataFrame(dataList):
    # Converts the DataList into a DataFrame.
    df = pd.DataFrame(dataList)
    return df

**convertDataFrametoCSV :**

**Takes a DataFrame and a file name as input, exports the DataFrame to a CSV file with the specified file name.**

In [ ]:
def convertDataFrametoCSV(fileName, dataFrame):
    # Exporting the Data Frame to CSV
    dataFrame.to_csv(str(fileName)+'.csv', index=False)

**convertAndExportDataAsJSON :**

**Converts a list of data into JSON format and exports it to a file named 'JSONoutput.json'.**

In [ ]:
def convertAndExportDataAsJSON(DataList):
    # Save JSON data to a file
    with open('JSONOutput.json', 'w') as json_file:
        json.dump(DataList, json_file, indent=4)

**getLikesVSViewsRatio :**

**Gets the likes vs views ratio of the top 10 Commented videos obtained.**

In [ ]:
def getLikesVSViewsRatio(VideoStatisticsData):
    try:
        # Loop through the Statistics Data.
        for Video in VideoStatisticsData:
            videoId = Video['Video_Id']
            likes = Video['Likes_count']
            views = Video['Views_Count']

            if views > 0:
                # Calculate the LikesvsViews Ratio
                ratio = likes/views
                print(f'The Likes Vs Views Ratio For VideoId: {videoId} is : {ratio:.5f},  with Likes: {likes}, Views:{views}')
            else:
                print(f'The Likes Vs Views Ratio For VideoId:{videoId} is : 0')
    except Exception as e:
           print(f'An Error has Occured While Calculating the Ratio: {e}')

# Execute the Required Set Of Commands

Testing whether the API Call works by Searching the videos related to the query string  “avatar movie”.

In [ ]:
try:
    # Getting Most Relevant Video Related Information In US Region.
    search_response = youtube_API.search().list(
        q=query_string,
        type='video',
        part='id,snippet',
        maxResults = 1,
        regionCode ='US',
        order = 'relevance'
    ).execute()

    # Performing Data Collection Steps.
    id = search_response['items'][0]['id']
    snippet = search_response['items'][0]['snippet']
    channelId = search_response['items'][0]['snippet']['channelId']
    channelTitle = search_response['items'][0]['snippet']['channelTitle']
    video_title = search_response['items'][0]['snippet']['title']
    video_description = search_response['items'][0]['snippet']['description']

    # Printing the Requested Video Information.
    print(f'Id :{id}')
    print('\n')
    print('Required Attributes From the Snippet : ')
    print(f'Channel Id :{channelId}')
    print(f'Channel Title: {channelTitle}')
    print(f'Video Title: {video_title}')
    print(f'Video Description : {video_description}')

    print('\n')
    print("Entire Snippet for Validation:")
    pprint(snippet)


except HttpError as e:
       print(f'An Error has Occured While Collecting the Data For the Video of Choice: {e}')


Id :{'kind': 'youtube#video', 'videoId': 'ByAn8DF8Ykk'}


Required Attributes From the Snippet : 
Channel Id :UCWOA1ZGywLbqmigxE4Qlvuw
Channel Title: Netflix
Video Title: Avatar: The Last Airbender | Official Trailer | Netflix
Video Description : MASTER YOUR ELEMENT. Avatar: The Last Airbender the live-action series arrives on February 22 only on Netflix. Watch on ...


Entire Snippet for Validation:
{'channelId': 'UCWOA1ZGywLbqmigxE4Qlvuw',
 'channelTitle': 'Netflix',
 'description': 'MASTER YOUR ELEMENT. Avatar: The Last Airbender the '
                'live-action series arrives on February 22 only on Netflix. '
                'Watch on ...',
 'liveBroadcastContent': 'none',
 'publishTime': '2024-01-23T15:58:10Z',
 'publishedAt': '2024-01-23T15:58:10Z',
 'thumbnails': {'default': {'height': 90,
                            'url': 'https://i.ytimg.com/vi/ByAn8DF8Ykk/default.jpg',
                            'width': 120},
                'high': {'height': 360,
                      

Gathering statistics for query string “avatar movie” of top 50 videos sorted by relevance in the US region.


In [ ]:
# Gets the Information related to top 50 movies in the US region ordered by relevance.
videoInformationOfTOP50Videos = getTopFiftyVideosDataOrderedByRelevance(query_string)

for videoInformation in videoInformationOfTOP50Videos:
    # Gather Statstic Details of Each Video.
    Video_Statistics= getVideoSpecificStatistics(videoInformation['video_id'])

    # Gather Videos With Comments Disabled to Filter out from the Top Commented Videos.
    DisabledComments_Videos = getVideoIdsWithDisabledComments(videoInformation['video_id'])

# Converting the Statistics Data to DataFrame.
statisticsOfTOP50Videos = getDataFrame(Video_Statistics)

# Converts the DataFrame to a CSV and Creates a Folder to Store it.
convertDataFrametoCSV('TOP 50 Videos Statistics', statisticsOfTOP50Videos)

In [ ]:
# Filter Out the Video's With Disabled Comments to Mitigate 403 Errors.
filteredVideoStatistics = [item for item in Video_Statistics if item['Video_Id'] not in DisabledComments_Videos]

Sort the data 2.b  by top 10 comments in descending order and consider the video IDs and Titles of top 10 videos which have highest comments.

In [ ]:
#Sort the Video Statistics Based on Comment Count in Descending Order
videosSortedByComments = sorted(filteredVideoStatistics, key=lambda x: x['Comments_count'], reverse=True)

#Top 10 Videos Based on Comments
top10CommentedVideos =videosSortedByComments[:10]

# Get the VideoID and Video Title for the TOP 10 Videos.
top10CommentedVideosInfo = [{'Video_Id': video['Video_Id'], 'Video_Title':video['Video_title']} for video in top10CommentedVideos]

Use a suitable method to retrieve comments of those top 10 videos from above For doing this, write a program to loop through each video id from 3.a and pass in the part parameter set to "snippet", to retrieve basic details about the comments. Execute this request and print the response using the pprint() method.

In [ ]:
# Loop through the TOP 10 Videos to Get their Comments.
for video in top10CommentedVideosInfo:
    Video_Comments= getCommentsFromVideo(video['Video_Id'])

# Print the Comments of Top 10 Videos.
pprint(Video_Comments)

[{'Video_Comments': [{'CommentInfo': {'etag': 'RVDjFIYMEfAOuOcFH47eqtYNBH0',
                                      'id': 'UgyT8ns0iaM-XvEu76l4AaABAg',
                                      'kind': 'youtube#commentThread',
                                      'snippet': {'canReply': True,
                                                  'channelId': 'UCgjxQJ6TlKqhHax8742ZMdA',
                                                  'isPublic': True,
                                                  'topLevelComment': {'etag': 'J0P26oge-IolAh41Nw4pNKZxuQM',
                                                                      'id': 'UgyT8ns0iaM-XvEu76l4AaABAg',
                                                                      'kind': 'youtube#comment',
                                                                      'snippet': {'authorChannelId': {'value': 'UCZt_npcoDicln_zzBOKS23g'},
                                                                                  'authorChannelUrl

Exporting the output of the above in JSON file format.

In [ ]:
# Converting the List to a JSON.
convertAndExportDataAsJSON(Video_Comments)

Function to get  the likes vs views ratio of the top 10 videos obtained above with the highest comments.

In [ ]:
# Get the Likes Vs Views Ratio for the TOP 10 Commented Videos.
getLikesVSViewsRatio(top10CommentedVideos)

The Likes Vs Views Ratio For VideoId: d9MyW72ELq0 is : 0.01779,  with Likes: 1042369, Views:58588946
The Likes Vs Views Ratio For VideoId: waJKJW_XU90 is : 0.02277,  with Likes: 459600, Views:20188824
The Likes Vs Views Ratio For VideoId: a8Gx8wiNbs8 is : 0.02440,  with Likes: 682179, Views:27963637
The Likes Vs Views Ratio For VideoId: 2r71I8lvTIA is : 0.03005,  with Likes: 156551, Views:5209436
The Likes Vs Views Ratio For VideoId: ByAn8DF8Ykk is : 0.03913,  with Likes: 277256, Views:7086156
The Likes Vs Views Ratio For VideoId: MgV9vymLIdQ is : 0.01876,  with Likes: 116745, Views:6222799
The Likes Vs Views Ratio For VideoId: -egQ79OrYCs is : 0.00209,  with Likes: 22297, Views:10663313
The Likes Vs Views Ratio For VideoId: bDHD1ueL4a4 is : 0.02192,  with Likes: 650931, Views:29698370
The Likes Vs Views Ratio For VideoId: 5PSNL1qE6VY is : 0.00633,  with Likes: 81729, Views:12917579
The Likes Vs Views Ratio For VideoId: RGx8rYbRVR4 is : 0.05614,  with Likes: 31089, Views:553746
